# Data imputation

## Time series

In [3]:
import impyute
from tqdm.notebook import tqdm

In [2]:
from imblearn.over_sampling import ADASYN, SMOTE
from sklearn.feature_selection import SelectKBest, f_regression, chi2, f_classif
from sklearn.metrics import f1_score, mean_squared_error, accuracy_score, r2_score, roc_auc_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression, BayesianRidge, LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import Lasso
from sklearn.svm import SVC, LinearSVC

import xgboost as xgb

In [4]:
import pandas as pd
import numpy as np
import torch

In [5]:
IDENTIFIERS = ["pid", "Time", "Age"]
MEDICAL_TESTS = [
    "LABEL_BaseExcess",
    "LABEL_Fibrinogen",
    "LABEL_AST",
    "LABEL_Alkalinephos",
    "LABEL_Bilirubin_total",
    "LABEL_Lactate",
    "LABEL_TroponinI",
    "LABEL_SaO2",
    "LABEL_Bilirubin_direct",
    "LABEL_EtCO2",
]
VITAL_SIGNS = ["LABEL_RRate", "LABEL_ABPm", "LABEL_SpO2", "LABEL_Heartrate"]
SEPSIS = ["LABEL_Sepsis"]
ESTIMATOR = {"bayesian": BayesianRidge(), "decisiontree": DecisionTreeRegressor(max_features="sqrt", random_state=0), 
                "extratree": ExtraTreesRegressor(n_estimators=10, random_state=0), 
                "knn": KNeighborsRegressor(n_neighbors=10, weights="distance")}
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


In [6]:
def sigmoid_f(x):
    """To get predictions as confidence level, the model predicts for all 12 sets of measures for
    each patient a distance to the hyperplane ; it is then transformed into a confidence level using
    the sigmoid function ; the confidence level reported is the mean of all confidence levels for a
    single patient

    Args:
        x (float): input of the sigmoid function

    Returns:
       float: result of the sigmoid computation.

    """
    return 1 / (1 + np.exp(-x))

In [12]:
# download data 
df_train = pd.read_csv(r"data/train_features.csv")
df_train_label = pd.read_csv(r"data/train_labels.csv")
df_val = pd.read_csv(r"data/test_features.csv")

In [ ]:
df_train_mean = df_train.copy().fillna(df_train.mean())
df_test_mean = df_val.copy().fillna(df_val.mean())

In [7]:
PRESENCE_IND = ['EtCO2','Lactate','Bilirubin_direct', 'Bilirubin_total', 'TroponinI',  'PaCO2', 'FiO2', 'AST', 'SaO2']
MEAN_IND= ['BUN', 'PTT', 'Hgb', 'HCO3', 'Alkalinephos', 'Chloride', 'Hct', 'Fibrinogen', 'Phosphate', 'WBC', 'Creatinine', 
              'Platelets', 'Glucose', 'Magnesium', 'Potassium', 'Calcium', 'pH', 'BaseExcess']
TIME_IND = ['Heartrate', 'Temp', 'SpO2', 'ABPs', 'RRate', 'ABPm', 'ABPd']

From visual analysis, it seems like we should treat the columns in the following way:
* EtCO2, Lactate, Bilirubin_direct, Bilirubin_total, TroponinI, PaCO2, FiO2, AST, SaO2 : presence (etco2 for acute respiratory distress, lactate is indicator of sepsis shock, bilirubin for liver diseases, TroponinI if you've had heart issues, etc)
* BUN, PTT, Hgb, HCO3, Alkalinephos, chloride, hct, fibrinogen, phosphate, WBC, creatinine, platelet, glucose, magnesium, potassium, calcium: set all missing values to mean of existing values (these are all blood test indicators)
* pH, base excess : set all missing values to mean of existing values (these values should be fully correlated)
* Heartrate, Temp, SpO2, ABPs, RRate, ABPm, ABPd : time series imputation, LCOF 


In [ ]:
df_train_copy = df_train.copy()

In [16]:
def transform_df(df, PRESENCE_IND, MEAN_IND, TIME_IND):
    """Performs data imputation and feature transformation in the following manner : for all data in PRESENCE_IND, 
    adds a column with the presence or absence for each patient of a test feature; for all data in PRESENCE_IND and MEAN_IND,
    if there are any values for a patient, replaces the NaN by the mean of that patient; for data in TIME_IND, does linear 
    interpolation of values to fill the time series
    
    Args: df (pandas.core.DataFrame): dataframe to transform
        PRESENCE_IND (list): features for presence transformation
        MEAN_IND (list): features for mean transformation
        TIME_IND (list): features for time series interpolation
    
    Returns: df (pandas.core.DataFrame): transformed dataframe
    """
    for pid in tqdm(df["pid"].unique()):
        pdf = df.loc[df["pid"]==pid]
        for col in PRESENCE_IND:
            if not(pd.isnull(pdf[col]).all()):
                df.loc[df["pid"]==pid, col+"_presence"]=1
            else:
                df.loc[df["pid"]==pid, col+"_presence"]=0
        for col in MEAN_IND+PRESENCE_IND:
            if not(pd.isnull(pdf[col]).all()):
                df.loc[df["pid"]==pid, col]=df.loc[df["pid"]==pid, col].fillna(df.loc[df["pid"]==pid, col].mean())
        for col in TIME_IND:
            if not(pd.isnull(pdf[col]).all()):
                df.loc[df["pid"]==pid, col]=df.loc[df["pid"]==pid, col].interpolate()
                df.loc[df["pid"]==pid, col]=df.loc[df["pid"]==pid, col].fillna(method='ffill')
                df.loc[df["pid"]==pid, col]=df.loc[df["pid"]==pid, col].fillna(method='bfill')
    return df     

In [ ]:
# transform the training data
df_train_copy = transform_df(df_train_copy, PRESENCE_IND, MEAN_IND, TIME_IND)

In [ ]:
df_train_copy.to_csv("df_train_new_imputation.csv", float_format="%.3f", index=False)

In [11]:
df_train_copy = pd.read_csv("df_train_new_imputation.csv")
df_train_copy

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,pH,EtCO2_presence,Lactate_presence,Bilirubin_direct_presence,Bilirubin_total_presence,TroponinI_presence,PaCO2_presence,FiO2_presence,AST_presence,SaO2_presence
0,1,3,34.0,NaN,NaN,12.0,NaN,36.0,8.700,24.000,...,7.33,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,1,4,34.0,NaN,NaN,12.0,NaN,36.0,8.567,25.333,...,7.33,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,1,5,34.0,NaN,NaN,12.0,NaN,36.0,8.567,25.333,...,7.37,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1,6,34.0,NaN,NaN,12.0,NaN,37.0,8.567,25.333,...,7.37,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,1,7,34.0,NaN,NaN,12.0,NaN,37.0,8.567,25.333,...,7.41,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227935,9999,8,85.0,NaN,36.4,30.0,NaN,36.0,9.450,25.000,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
227936,9999,9,85.0,NaN,36.4,30.0,NaN,36.0,9.450,25.000,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
227937,9999,10,85.0,NaN,36.4,30.0,NaN,36.0,9.450,25.000,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
227938,9999,11,85.0,NaN,36.4,30.0,NaN,36.0,10.200,25.000,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
# construct the training labels
df_train_label_copy = df_train_label.set_index(["pid"])
df_label = df_train_label_copy.loc[df_train_copy["pid"].unique().astype(int)]
df_label = df_label.reset_index()
df_label

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.1,85.4,100.0,59.9
1,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8,100.6,95.5,85.5
2,100,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16.5,88.3,96.5,108.1
3,1000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,19.4,77.2,98.3,80.9
4,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.6,76.8,97.7,95.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,17.1,69.8,100.0,110.7
18991,9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.6,97.3,97.8,59.2
18992,9996,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,17.3,66.3,96.9,100.3
18993,9998,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,18.8,81.5,96.9,99.4


In [14]:
df_test_copy = df_val.copy().head(6000)

In [17]:
df_test_copy = transform_df(df_test_copy, PRESENCE_IND, MEAN_IND, TIME_IND)

Then, we take the median of all these values for the 12h of the patient to get one line per patient for tasks 1 & 2, and keep all values for task 3

In [18]:
df_train_1_2 = df_train_copy.groupby(["pid"], as_index=False).median()
df_train_3 = df_train_copy[IDENTIFIERS + TIME_IND]
df_test_1_2 = df_test_copy.groupby(["pid"], as_index=False).median()
df_test_3 = df_test_copy[IDENTIFIERS + TIME_IND]

Finally, we use a data imputer to solve values that are still NaN

In [19]:
def perform_imputation(df_train,df_test):
    columns = df_train.columns
    imputer = IterativeImputer(
        estimator=KNeighborsRegressor(),
        missing_values=np.nan,
        sample_posterior=False,
        max_iter=10,
        tol=0.001,
        n_nearest_features=None, # Meaning all features are used
        initial_strategy='median',
        imputation_order='descending',
        skip_complete=False,
        min_value=None,
        max_value=None,
        verbose=2,
        random_state=42,
        add_indicator=False
    )

    df_train = imputer.fit_transform(df_train.values)
    df_train = pd.DataFrame(df_train, columns=columns)
    df_train['pid'] = df_train['pid'].astype(int)
    df_test = imputer.transform(df_test.values)
    df_test = pd.DataFrame(df_test, columns=columns)
    df_test['pid'] = df_test['pid'].astype(int)
    return df_train,df_test

In [20]:
df_train_1_2, df_test_1_2 = perform_imputation(df_train_1_2, df_test_1_2)
df_train_1_2.to_csv("df_train_1_2.csv", float_format="%.3f", index=False)

[IterativeImputer] Completing matrix with shape (18995, 46)
[IterativeImputer] Ending imputation round 1/10, elapsed time 22.03
[IterativeImputer] Change: 2341.6249, scaled tolerance: 31.658 
[IterativeImputer] Ending imputation round 2/10, elapsed time 45.76
[IterativeImputer] Change: 2009.5840999999998, scaled tolerance: 31.658 
[IterativeImputer] Ending imputation round 3/10, elapsed time 69.45
[IterativeImputer] Change: 1785.1525000000001, scaled tolerance: 31.658 
[IterativeImputer] Ending imputation round 4/10, elapsed time 94.44
[IterativeImputer] Change: 2038.6158, scaled tolerance: 31.658 
[IterativeImputer] Ending imputation round 5/10, elapsed time 117.93
[IterativeImputer] Change: 2002.0263, scaled tolerance: 31.658 
[IterativeImputer] Ending imputation round 6/10, elapsed time 141.02
[IterativeImputer] Change: 2007.9604999999997, scaled tolerance: 31.658 
[IterativeImputer] Ending imputation round 7/10, elapsed time 165.02
[IterativeImputer] Change: 1900.2678, scaled toler

C:\Users\josep\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


[IterativeImputer] Ending imputation round 2/10, elapsed time 0.34
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.50
[IterativeImputer] Ending imputation round 4/10, elapsed time 0.66
[IterativeImputer] Ending imputation round 5/10, elapsed time 0.81
[IterativeImputer] Ending imputation round 6/10, elapsed time 0.97
[IterativeImputer] Ending imputation round 7/10, elapsed time 1.12
[IterativeImputer] Ending imputation round 8/10, elapsed time 1.28
[IterativeImputer] Ending imputation round 9/10, elapsed time 1.43
[IterativeImputer] Ending imputation round 10/10, elapsed time 1.59


In [ ]:
df_train_3, df_test_3 = perform_imputation(df_train_3, df_test_3)

In [21]:
def data_formatting(df_train, df_test, df_label):
    # Data formatting
    X_train= df_train.drop(columns=["pid","Time"]).values
    X_test = df_test.drop(columns=["pid","Time"]).values
    # Create list with different label for each medical test
    print("Creating a list of labels for each medical test")
    y_train_medical_tests = []
    for test in MEDICAL_TESTS:
        y_train_medical_tests.append(df_label[test].astype(int).values)

    # Create list with different label for sepsis
    print("Creating a list of labels for sepsis")
    y_train_sepsis = []
    for sepsis in SEPSIS:
        y_train_sepsis.append(df_label[sepsis].astype(int).values)
    return X_train,X_test,y_train_medical_tests,y_train_sepsis

In [22]:
X_train_1_2, X_test_1_2, y_train_medical_tests,y_train_sepsis = \
data_formatting(df_train_1_2,df_test_1_2,df_label)

Creating a list of labels for each medical test
Creating a list of labels for sepsis


In [ ]:
X_train_mean, X_test_mean, y_train_medical_tests_mean, y_train_sepsis_mean = \
    data_formatting(df_train_mean,df_test_mean,df_train_label)

In [ ]:
for i in range(len(y_train_medical_tests_mean)):
    newy = []
    for j in range(len(y_train_medical_tests_mean[i])):
        newy+=[y_train_medical_tests_mean[i][j] for k in range(12)]
    y_train_medical_tests_mean[i]=np.array(newy)

# Task 1

In [23]:
# Scale data 
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_scaled = scaler.fit_transform(X_train_1_2)
X_test_scaled = scaler.transform(X_test_1_2)

In [ ]:
# Scale data 
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_scaled_mean = scaler.fit_transform(X_train_mean)
X_test_scaled_mean = scaler.transform(X_test_mean)

In [24]:
# Modelling of medical tests using SVC
models = []
scores = []
columns_medical_tests = []
for i, test in enumerate(MEDICAL_TESTS):
    X_train, X_test, y_train, y_test = train_test_split(
        X_train_scaled, y_train_medical_tests[i], test_size=0.20, random_state=42, shuffle=True
    )
    
    print("Resampling data")
    sampler = ADASYN()
    X_train, y_train = sampler.fit_sample(X_train, y_train)
    print(f"Fitting model for {test}.")


    feature_selector = SelectKBest(score_func=f_classif, k=3)
    X_train = feature_selector.fit_transform(X_train, y_train)
    X_test = feature_selector.transform(X_test)
    columns = feature_selector.get_support(indices=True)
    columns_medical_tests.append(columns)

    clf = SVC(C=0.01, kernel="poly", degree=4, class_weight="balanced").fit(X_train, y_train)
    models.append(clf)
    y_pred = clf.decision_function(X_test)
    y_pred = [sigmoid_f(y_pred[i]) for i in range(len(y_pred))]
    scores.append(roc_auc_score(y_test, y_pred))
print(f"Scores {scores}")
print(f"Average score {np.mean(scores)}")

Resampling data
Fitting model for LABEL_BaseExcess.
Resampling data
Fitting model for LABEL_Fibrinogen.


C:\Users\josep\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


Resampling data
Fitting model for LABEL_AST.
Resampling data
Fitting model for LABEL_Alkalinephos.
Resampling data
Fitting model for LABEL_Bilirubin_total.


C:\Users\josep\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


Resampling data
Fitting model for LABEL_Lactate.
Resampling data
Fitting model for LABEL_TroponinI.
Resampling data
Fitting model for LABEL_SaO2.
Resampling data
Fitting model for LABEL_Bilirubin_direct.


C:\Users\josep\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


Resampling data
Fitting model for LABEL_EtCO2.
Scores [0.5159676021079085, 0.49051554228528216, 0.49590452377294636, 0.4904578391811426, 0.5057996351762872, 0.5110523259048464, 0.5112683811621429, 0.5104214473076322, 0.47246367696081515, 0.4959382493034378]
Average score 0.49997892231624413


C:\Users\josep\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


In [ ]:
# Modelling of medical tests using logistic regression with cross validation
models = []
scores = []
columns_medical_tests = []
for i, test in enumerate(MEDICAL_TESTS):
    X_train, X_test, y_train, y_test = train_test_split(
        X_train_scaled, y_train_medical_tests[i], test_size=0.20, random_state=42, shuffle=True
    )
    
    print("Resampling data")
    sampler = ADASYN()
    X_train, y_train = sampler.fit_sample(X_train, y_train)
    print(f"Fitting model for {test}.")

    feature_selector = SelectKBest(score_func=f_classif, k=3)
    X_train = feature_selector.fit_transform(X_train, y_train)
    X_test = feature_selector.transform(X_test)
    columns = feature_selector.get_support(indices=True)
    columns_medical_tests.append(columns)
    clf = LogisticRegressionCV(cv=5, random_state=42).fit(X_train, y_train)
    models.append(clf)
    scores.append(roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]))
print(f"Scores {scores}")
print(f"Average score {np.mean(scores)}")

Resampling data
Fitting model for LABEL_BaseExcess.
Resampling data
Fitting model for LABEL_Fibrinogen.
Resampling data
Fitting model for LABEL_AST.
Resampling data
Fitting model for LABEL_Alkalinephos.
Resampling data
Fitting model for LABEL_Bilirubin_total.
Resampling data
Fitting model for LABEL_Lactate.
Resampling data
Fitting model for LABEL_TroponinI.
Resampling data


In [ ]:
# using XGBoost
clf = xgb.XGBClassifier(objective="binary:logistic", n_thread=-1)

scores = []
for i, test in enumerate(MEDICAL_TESTS):
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_train_scaled, y_train_medical_tests[i], test_size=0.20, random_state=42, shuffle=True
    )
    
    print("Resampling data")
    sampler = ADASYN()
    X_train, y_train = sampler.fit_sample(X_train, y_train)
    print(f"Fitting model for {test}.")


    label_counts = pd.Series(y_train).value_counts()
    scale_pos_weight = label_counts[0] / label_counts[1]
    if scale_pos_weight < 2: scale_pos_weight = 2
    
    # Coarse parameter grid not optimized at all yet
    param_grid = {
        "booster": ["dart"],
        "eta": [0.1,0.01, 0.001],
        "min_child_weight": range(1, 10, 1),
        "max_depth": range(3, 8, 1),
        "gamma": range(0, 100, 2),
        "max_delta_step": range(3, 10, 1),
        "subsample": np.arange(0.1, 1, 0.05),
        "colsample_bytree": np.arange(0.3, 1, 0.05),
        "n_estimators": range(50, 120, 2),
        "scale_pos_weight": [int(scale_pos_weight)],
        "reg_lambda": [100], # Ridge regularization
        "reg_alpha": [0], # Lasso regularization
        "eval_metric": ["error"]
    }
    
    feature_selector = SelectKBest(score_func=f_classif, k=3)
    X_train = feature_selector.fit_transform(X_train, y_train)
    X_test = feature_selector.transform(X_test)
    columns = feature_selector.get_support(indices=True)
    columns_medical_tests.append(columns)

    coarse_search = RandomizedSearchCV(estimator=clf,
            param_distributions=param_grid, scoring="roc_auc",
            n_jobs=-1, cv=5, n_iter=10, verbose=1)
    coarse_search.fit(X_train, y_train)
    models.append(coarse_search.best_estimator_)
    scores.append(roc_auc_score(y_test, coarse_search.best_estimator_.predict_proba(X_test)[:,1]))
print(f"Scores {scores}")
print(f"Average score {np.mean(scores)}")

# Task 2

In [ ]:
# Model and predict sepsis

X_train, X_test, y_train, y_test = train_test_split(
    X_train_scaled, y_train_sepsis[0], test_size=0.20, random_state=42, shuffle=True
)
print("Resampling data")
sampler = ADASYN()
X_train, y_train = sampler.fit_sample(X_train, y_train)


print("Applying feature selection")
feature_selector = SelectKBest(score_func=f_classif, k=3)
X_train = feature_selector.fit_transform(X_train, y_train)
X_test = feature_selector.transform(X_test)
columns_sepsis = feature_selector.get_support(indices=True)

models, scores = [], []
for i, C in enumerate(np.linspace(0.0001, 0.01, num=5)):
    print("Starting SVC for C={}".format(C))
    models.append(
        SVC(C=C, kernel="poly", class_weight="balanced").fit(X_train, y_train)
    )
    y_pred = models[i].decision_function(X_test)
    y_pred = [sigmoid_f(y_pred[i]) for i in range(len(y_pred))]
    scores.append(roc_auc_score(y_test, y_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, np.around(y_pred)).ravel()
    size = len(y_test)
    print(
        "The ROC AUC score is {}, the TPrate is {}%, the FPrate is {}% "
        "for iteration {}".format(scores[i], tp/(tp+fn)*100, tn/(tn+fp)*100, i)
    )
    print(y_test[:30],np.around(y_pred[:30]))
best = np.argmax(scores)
model, score = models[best], scores[best]

# Task 3

In [ ]:
TREND_LABEL = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

In [ ]:
pd.set_option('precision', 3)

In [ ]:
def train_lasso(df_train_3, df_train_label, test, label_test, alpha):
    X_train, y_train = [],[]
    for pid in df_train_3["pid"].unique():
        X_train.append(df_train_3.loc[df_train_3["pid"]==pid, test].values)
        y_train.append(df_train_label.loc[df_train_label["pid"]==pid, label_test].values)
    X_train, y_train = np.array(X_train),np.array(y_train)
    X_train, X_test, y_train, y_test = train_test_split(
        X_train, y_train, test_size=0.20, random_state=42, shuffle=True
    )
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train, y_train)
    print("R2 score for lasso model using innate scoring: ",lasso_model.score(X_train, y_train))
    y_pred = lasso_model.predict(X_test)
    print("R2 score on test set : ",r2_score(y_test, y_pred))
    print(f"Diff is {np.around(np.abs([(y_pred[i]-y_test[i])/y_test[i] for i in range(len(y_pred))][:10]),2)}")
    return lasso_model

In [ ]:
def predict_lasso(lasso_model, df_test_3, test):
    X_test = []
    for pid in df_test_3["pid"].unique():
        X_test.append(df_test_3.loc[df_test_3["pid"]==pid, test].values)
    predictions = lasso_model.predict(X_test)
    return predictions

In [ ]:
def lasso_subtask_3(df_train_3, df_train_label, TREND_LABEL):
    lasso_models = []
    for label_test in TREND_LABEL:
        print('Testing model for {}'.format(label_test))
        test = label_test.split("_")[1]
        lasso_models.append(train_lasso(df_train_3, df_train_label, test, label_test, 0.0001))
    return lasso_models

In [ ]:
lasso_models = lasso_subtask_3(df_train_3, df_label, TREND_LABEL)

In [ ]:
def get_predictions_3(df_test_3, lasso_models, TREND_LABEL):
    predictions = []
    for i in range(len(lasso_models)):
        test = TREND_LABEL[i].split("_")[1]
        predictions.append(predict_lasso(lasso_models[i], df_test_3, test))
    predictions = np.array(predictions).T
    predictions = pd.DataFrame(data=predictions, columns=TREND_LABEL, index=(df_test_3["pid"].unique()).astype(int))
    return(predictions)

In [ ]:
predictions = get_predictions_3(df_test_3, lasso_models, TREND_LABEL)

In [ ]:
predictions

Discoveries: we already have the values of some we want to predict in the test set, so time series modelling? Prophet API possibly?